# Transformer 자료
# https://wikidocs.net/166802

In [1]:
# https://wikidocs.net/166802
import torch
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else 'cpu'# the device to load the model onto'
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained("distilbert-base-uncased", torch_dtype=torch.float32)#, attn_implementation="flash_attention_2")

text = "Hello"
encoded_input = tokenizer(text, return_tensors='pt').to(device)
print(encoded_input)

text = "world"
encoded_input = tokenizer(text, return_tensors='pt').to(device)
print(encoded_input)


text = "woprslxd"
encoded_input = tokenizer(text, return_tensors='pt').to(device)
print(encoded_input)

text = "Hello world [MASK]"
encoded_input = tokenizer(text, return_tensors='pt').to(device)
print(encoded_input)

model.to(device)

output = model(**encoded_input)
output.last_hidden_state.shape

# 101은 CLS, 1o2 = SEP, 103 = mask, UNK =  unkonw

# tokenizer :
# Byte-level BPE (GPT2)
# WorldPiece (BERT)

# DECODING
decoded_string = tokenizer.decode([101, 103 , 102])

print(decoded_string)

/Users/juniverse/opt/anaconda3/envs/0u0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 17.5kB/s]
config.json: 100%|██████████| 483/483 [00:00<00:00, 773kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.17MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.16MB/s]
model.safetensors: 100%|██████████| 268M/268M [00:23<00:00, 11.6MB/s] 


{'input_ids': tensor([[ 101, 7592,  102]]), 'attention_mask': tensor([[1, 1, 1]])}
{'input_ids': tensor([[ 101, 2088,  102]]), 'attention_mask': tensor([[1, 1, 1]])}
{'input_ids': tensor([[  101, 24185, 18098, 14540,  2595,  2094,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[ 101, 7592, 2088,  103,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
[CLS] [MASK] [SEP]


In [57]:
sequence = "I've been waiting course my whole life."
sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
print('tokens',tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print('ids',ids)
input_ids = torch.tensor(ids) # 이렇게 하면 안됨.

input_ids = torch.tensor([ids]) # 기본적으로 batching을 통해 여러 문장을 입력해야함.
batch_ids = [ids,ids] # batch 처리를 통해 모델이 여러문장을 동시에 입력받을수있도록 할수있음.
# 다만 이런경우에 직사각형 형태의 텐서가 되자않으므로,padding 필요. padding token추가

# 대부분의 모델은 512 ~1024개의 토큰 시퀀스 지원. 또한 길이가 더 긴 시퀀스는 절단(truncation)
# max_sequence_length = 512로 해서 절단하자!

print('input ids',input_ids)
# This line will fail

# input_ids = tokenizer(sequence, return_tensors='pt').to(device)
y = model(input_ids)
y.last_hidden_state.shape

tokens ['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
ids [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
input ids tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])


torch.Size([1, 14, 768])

In [62]:
print(tokenizer.pad_token_id)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
print(model_inputs)

# 해당 시퀀스를 리스트 내의 최대 시퀀스 길이까지 패딩(padding) 합니다.
model_inputs = tokenizer(sequences, padding="longest")

# 시퀀스를 모델 최대 길이(model max length)까지 패딩(padding) 합니다.
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# 지정된 최대 길이까지 시퀀스를 패딩(padding) 합니다.
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

# Truncation
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# 모델 최대 길이(model max length)보다 긴 시퀀스를 자릅니다.
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# 지정된 최대 길이보다 긴 시퀀스를 자릅니다.
model_inputs = tokenizer(sequences, max_length=8, truncation=True)




0
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


# Special Tokens

101, 102는 CLS, SEP token

In [63]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)




[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [70]:
# return_tensors는 np와 pt가 있음.

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="np")
print(tokens.input_ids.dtype)
print(type(tokens.input_ids))
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
print(tokens.input_ids.dtype)
print(type(tokens.input_ids))
output = model(**tokens)
print(output.last_hidden_state.shape)

int64
<class 'numpy.ndarray'>
torch.int64
<class 'torch.Tensor'>
torch.Size([2, 16, 768])


In [37]:
from transformers import BertConfig, BertModel

# config(설정)을 만듭니다.
config = BertConfig()

# 해당 config에서 모델을 생성합니다.
model = BertModel(config)
print(config)
config.hidden_act

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



'gelu'

# 데이터 처리작업 & 학습 시키기

In [72]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# 2장의 예제와 동일합니다.
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, torch_dtype=torch.float32)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# 새롭게 추가된 코드입니다.
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 허브에서 데이터셋 로딩

아래 결과에서 보듯이, 학습(training), 검증(validation) 및 평가(test) 집합이 저장된 DatasetDict 객체를 얻을 수 있습니다. 이들 각각은 여러 종류의 열(columns)(sentence1, sentence2, label 및 idx)과 행(row)의 개수를 포함하는데, 여기서 행(row)의 개수는 각 집합의 문장쌍의 개수를 나타냅니다. 따라서, 학습 집합(training set)에는 3,668개의 문장 쌍, 검증 집합(validation set)에는 408개, 평가 집합(test set)에는 1,725개의 문장 쌍이 있습니다.

load_dataset 명령은 기본적으로 ~/.cache/huggingface/dataset에 데이터셋을 다운로드하고 임시저장(캐시, cache)합니다. 2장에서 보았듯이, HF_HOME 환경 변수를 설정하여 캐시 폴더를 변경할 수 있습니다.

파이썬의 딕셔너리(dictionary)와 같이 키값으로 raw_datasets 개체의 개별 집합(학습, 검증, 평가)에 접근할 수 있습니다:

In [74]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [76]:
raw_train_dataset = raw_datasets["train"]
print(raw_train_dataset[0])
raw_train_dataset.features




{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [86]:
# dataset preprocessing  : using Tokenizer

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
print(len(tokenized_sentences_1.input_ids))

print(tokenized_sentences_1.input_ids[0])

# DECODING
decoded_string = tokenizer.decode(tokenized_sentences_1.input_ids[0])

print(decoded_string)


3668
[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102]
[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP]


In [89]:
# token_type_ids를 사용하는경우 ! distil bert는 이것이 없음. pretrain시에 이걸 입력으로 사용한 경우만 존재함.
# MLM 외에 next sentence prediction이라는 추가 object가 있음. 이 작업의 목표는 문장간의 관계를 모델링하는것.
# 하지만 논문에서 성능에 영향이 없다고 하므로 깔끔하게 제거하자!

tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)
tokenized_dataset[0]

Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

>이 방법은 잘 작동하지만, input_ids, attention_mask, token_type_ids 및 데이터가 담겨진 다차원 리스트가 키로 지정된tokenized_dataset이라는 별도의 파이썬 딕셔너리를 반환하는 단점이 있습니다. 또한 이 방법은 토큰화하는 동안 전체 데이터셋을 저장할 충분한 공간의 RAM이 있는 경우에만 작동합니다. 반면, 🤗Datasets 라이브러리의 데이터셋들은 디스크에 저장된 Apache Arrow 파일이므로, 요청한 샘플만 메모리에 로드된 상태로 유지합니다.

특정 데이터를 dataset 객체로 유지하기 위해 Dataset.map() 메서드를 사용합니다.

 이 방법은 토큰화(tokenization) 외에 더 많은 전처리가 필요한 경우 유연성을 발휘합니다. 
 
 map() 메서드는 데이터셋의 개별 요소에 함수(function)를 적용하여 작동하므로 입력을 토큰화하는 함수를 정의해 보겠습니다:


In [90]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


이 함수는 데이터셋의 개별 항목이 담겨진 딕셔너리를 매개변수로 입력받아서 input_ids, attention_mask 및 token_type_ids 키가 지정된 새로운 딕셔너리를 반환합니다. 이전에 본 것처럼 토크나이저(tokenizer)는 문장 쌍 리스트에서 작동하기 때문에 example 딕셔너리에 여러 샘플(각 키가 문장 목록임)이 포함된 경우에도 작동합니다. 이렇게 하면 map() 호출에서 batched=True 옵션을 사용할 수 있어 토큰화 속도가 크게 빨라집니다. 토크나이저는 🤗Tokenizers 라이브러리에서 Rust로 작성된 또 다른 토크나이저에 의해 지원됩니다. 이 토크나이저는 매우 빠를 수 있지만, 한 번에 많은 입력을 제공하는 경우에만 그렇습니다.

일단 현재는 토큰화 함수에서 padding 매개변수를 생략했습니다. 이는 모든 샘플들을 최대 길이로 채우는 것(padding)이 효율적이지 않기 때문입니다. 배치(batch) 형태로 실행할 때 샘플을 채우는 것(padding)이 효과를 발휘합니다. 그러면 전체 데이터셋에서의 최대 길이가 아니라 해당 배치(batch) 내에서의 최대 길이로 채우기만(padding) 하면 되기 때문입니다. 이것은 입력의 길이가 매우 가변적일 때 많은 시간과 처리 능력을 절약할 수 있습니다!

In [92]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

# 동적 패딩

샘플들을 함께 모아서 지정된 크기의 배치(batch)로 구성하는 역할을 하는 함수를 콜레이트 함수(collate function) 라고 합니다. 이 함수는 DataLoader를 빌드(build)할 때 전달할 수 있는 매개변수입니다. 기본값은 단순히 샘플들을 PyTorch 텐서로 변환하고 결합하는 함수입니다. 만일 대상 샘플들이 리스트, 튜플 혹은 딕셔너리면 재귀적으로 이 작업이 수행됩니다. 우리 예제의 경우, 입력값이 모두 동일한 크기(길이)가 아니기 때문에 이 작업이 불가능합니다. 지금까지 우리는 일부러 패딩(padding) 작업을 미뤄왔는데 그 이유는 전체 데이터셋이 아닌 개별 배치(batch)에 대해서 별도로 패딩(padding)을 수행하여 과도하게 긴 입력으로 인한 과도한 패딩(padding) 작업을 방지하기 위함입니다. 이렇게 하면 학습 속도가 상당히 빨라지지만 TPU에서 학습하는 경우 문제가 발생할 수 있습니다. TPU는 추가적인 패딩(padding)이 필요한 경우에도 전체 데이터셋이 고정된 형태를 선호합니다.

실제로 이를 수행하려면, 배치(batch)로 분리하려는 데이터셋의 요소 각각에 대해서 정확한 수의 패딩(padding)을 적용할 수 있는 콜레이트 함수(collate function)를 정의해야 합니다. 다행히도, 🤗Transformers 라이브러리는 DataCollatorWithPadding을 통해 이러한 기능을 제공합니다. 이 함수는 토크나이저를 입력으로 받습니다. 그 이유는 사용하려는 패딩 토큰(padding token)이 무엇인지와 모델이 입력의 왼쪽 혹은 오른쯕 중 어느 쪽에 패딩(padding)을 수행할지를 파악하기 위함입니다. 이 입력 하나면 모든 것이 해결됩니다:


In [95]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# 배치(batch) 내의 각 요소들의 길이를 살펴보세요:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]



[50, 59, 47, 67, 59, 50, 62, 32]

# trainer

In [100]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
python(2330) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
python(2331) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to av

  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

# Evaluation

In [102]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

# fine tuning

In [107]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint,torch_dtype = torch.float32)
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

text = "This is a great [MASK]."

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# [MASK]의 위치를 찾고, 해당 logits을 추출합니다.
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# 가장 큰 logits값을 가지는 [MASK] 후보를 선택합니다.
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")


'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'
'>>> This is a great deal.'
'>>> This is a great success.'
'>>> This is a great adventure.'
'>>> This is a great idea.'
'>>> This is a great feat.'


In [108]:
# movie semantics anaylsis
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [111]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")

# preprocessing
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# 빠른 멀티스레딩을 작동시키기 위해서, batched=True를 지정합니다.
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets



'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(tokenizer.model_max_length)



# BART

In [10]:
from transformers import BartConfig, BartModel

# Initializing a BART facebook/bart-large style configuration
configuration = BartConfig()

# Initializing a model (with random weights) from the facebook/bart-large style configuration
model = BartModel(configuration)

# Accessing the model configuration
configuration = model.config

In [23]:
from transformers import AutoTokenizer, BartModel
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartModel.from_pretrained("facebook/bart-base",torch_dtype=torch.float32)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

inputs = inputs.to(torch.float32)

outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Attempting to cast a BatchEncoding to type torch.float32. This is not supported.


RuntimeError: value cannot be converted to type at::Half without overflow

In [ ]:
# summarization
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
# quesion & answering
from transformers import AutoTokenizer, BartForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-finetuned-squadv1")
model = BartForQuestionAnswering.from_pretrained("valhalla/bart-large-finetuned-squadv1")

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)